# Prep dataset

In [ ]:
import pandas as pd
import numpy as np
import os
import scipy
import scipy.stats

from google.colab import drive
drive.mount('/content/drive')
data_foloder = 'here'
result = 'here'

baseline = pd.read_csv(data_foloder + 'here.csv')
incoh10 = pd.read_csv(data_foloder + 'here.csv')
incoh20 = pd.read_csv(data_foloder + 'here.csv')
incoh50 = pd.read_csv(data_foloder + 'here.csv')
ineff10 = pd.read_csv(data_foloder + 'here.csv')
ineff20 = pd.read_csv(data_foloder + 'here.csv')
ineff50 = pd.read_csv(data_foloder + 'here.csv')

baseline.head()

In [2]:
os.getcwd()

'/content'

# Install lib and dependencies

In [3]:
# install sentence-transformers from hugging face
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.6 MB/s 
     |████████████████████████████████| 5.5 MB 28.8 MB/s 
     |████████████████████████████████| 1.3 MB 55.1 MB/s 
     |████████████████████████████████| 182 kB 58.7 MB/s 
     |████████████████████████████████| 7.6 MB 39.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=023521ed55d95c03fd6587c3abfb1c0b197841ab9726360a7529954e2f309722
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [4]:
# Utilize BERT
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('stsb-roberta-large')
type(model) #if worked well, should print: sentence_transformers.SentenceTransformer.SentenceTransformer

Downloading:   0%|          | 0.00/748 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/191 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

sentence_transformers.SentenceTransformer.SentenceTransformer

# Contextualized sentence embeddings

In [ ]:
def get_sent_emb(input):
  sentence = input.split('.')[:-1]
  embedding = model.encode(sentence, convert_to_tensor=True)
  return embedding

# call the function 
# apply it to the dataframe
dfs = [baseline, incoh10, incoh20, incoh50, ineff10, ineff20, ineff50]
temp = -1
for df in dfs:
  temp += 1
  df["bert_sent_emb"] = ''
  df['bert_sent_emb'] = df['content'].apply(lambda x: get_sent_emb(x))
  df.to_csv(result + str(temp) + '_sent.csv')
df.head()

In [16]:
df.columns

Index(['grid', 'content', 'n_words', 'bert_sent_emb'], dtype='object')

# Stats and similarities functions

In [17]:
# stats ignoring nan, apply to all LMs 
from numpy import nanmedian

import scipy
def iqr(x):
  return scipy.stats.iqr(np.array(x), nan_policy='omit')

from numpy import nanquantile
def q5(x):
    return np.nanquantile(np.array(x), 0.05)

def q95(x):
    return np.nanquantile(np.array(x), 0.95)

In [18]:
# cosine_similarity, apply to all LMs
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# FOC and SOC

In [ ]:
cs=['foc', 'soc']
stats = ['_median', '_iqr', '_q5', '_q95']
for df in dfs:
    # create new empty columns
    for c in cs:
        for stat in stats:
            cur = 'bert_' + c + stat
            df[cur] = ''
df.head()

In [ ]:
temp_file = -1 # keep track of file names
for df in dfs:
    temp_file += 1
    df['bert_foc_similarity'] = ''
    df['bert_soc_similarity'] = ''
    # loop over each response
    for i in df.index:
        temp_foc = [] # for each individual, store the list of cos similarity
        temp_soc = []
        # calculate average similarity for sentence pairs, either adjacent or with one intervening
        for idx, sent in enumerate(df['bert_sent_emb'][i]):
            try:
                temp_foc.append(cosine_similarity(sent, df['bert_sent_emb'][i][idx+1])) # get a list of similarities for that response
                
                df['bert_foc_similarity'][i] = temp_foc # record intermediate similarities 
                
                df['bert_foc_median'][i] = np.nanmedian(temp_foc) # add more stats here
                df['bert_foc_iqr'][i] = iqr(temp_foc)
                df['bert_foc_q5'][i] = q5(temp_foc)
                df['bert_foc_q95'][i] = q95(temp_foc)

                temp_soc.append(cosine_similarity(sent, df['bert_sent_emb'][i][idx+2]))

                df['bert_soc_similarity'][i] = temp_soc # record intermediate similarities 

                df['bert_soc_median'][i] = np.nanmedian(temp_soc)
                df['bert_soc_iqr'][i] = iqr(temp_soc)
                df['bert_soc_q5'][i] = q5(temp_soc)
                df['bert_soc_q95'][i] = q95(temp_soc)

            except IndexError:
                continue
    df.to_csv(result + str(temp_file) + '_sent.csv')
df.head()